## 4.5 Überbestimmte Gleichungssysteme

Hier verwenden wir unsere bekannte QR-Zerlegung anhand von Householder-Transformationen, da diese bereits den nicht-quadratischen Fall abdeckt.

In [ ]:
import numpy as np
from scripts.LR_Zerlegung import rueckwaerts_einsetzen
from scripts.Cholesky import vorwaerts_einsetzen, cholesky

#### Beispiel 4.27 (Bestapproximation eines überbestimmten Gleichungssystems mit erweiterter QR-Zerlegung)

Wir betrachten das überbestimmte Gleichungssystem $Ax = b$ mit
$$
A = \begin{pmatrix}
  1 & -\frac{1}{4} & \frac{1}{16} \\
  1 & \frac{1}{2} & \frac{1}{4} \\
  1 & 2 & 4 \\
  1 & \frac{5}{2} &\frac{25}{4} 
\end{pmatrix},\quad b=\begin{pmatrix} 0 \\1 \\ 0 \\ 1
\end{pmatrix}.
$$

In [ ]:
A = np.array([[1, -1 / 4, 1 / 16],
              [1, 1 / 2,  1 / 4],
              [1, 2,      4],
              [1, 5 / 2,  25 / 4]], dtype=np.half)
b = np.array([0, 1, 0, 1], dtype=np.half)
A2 = A.copy()

Zunächst stellen wir das Normalensystem $A^T A = A^T b$ dazu auf

In [ ]:
ATA = A.transpose() @ A
ATb = np.inner(A.transpose(), b)

Und Lösen dies mit unserer Cholesky-Zerlegung

In [ ]:
L = cholesky(ATA)
y = vorwaerts_einsetzen(L, ATb)
x_L = rueckwaerts_einsetzen(L.transpose(), y)
print(x_L)

Hieraus ergibt sich der Defekt

In [ ]:
print('||Ax - b|| = ', np.linalg.norm(np.inner(A, x_L) - b))

Jetzt lösen wir das System nochmal anhand der QR-Zerlegung nach Householder. Da wir hierfür die modifizierte rechte Seite $\tilde{b}$ benötigen, müssen wir dies noch implementieren.

In [ ]:
from scripts.Householder import qr_householder

def b_tilde(V, b):
    bt = b.copy()
    m = A.shape[1]
    for i in range(m):
        bt[i:] -= 2 * np.inner(V[i:, i], bt[i:]) * V[i:, i]
    return np.array(bt[:m])

Damit erhalten wir, bei der Verwendung von `half` Gleitkommazahlen, die Lösung

In [ ]:
A = np.array([[1, -1 / 4, 1 / 16],
              [1, 1 / 2,  1 / 4],
              [1, 2,      4],
              [1, 5 / 2,  25 / 4]], dtype=np.half)
b = np.array([0, 1, 0, 1], dtype=np.half)

V = qr_householder(A)
bt = b_tilde(V, b)
x_h = rueckwaerts_einsetzen(A[:V.shape[1],:], bt)
print(f'x = {x_h}')

In [ ]:
print('||Ax - b|| = ', np.linalg.norm(np.inner(A2, x_h) - b))

Dabei haben wir jetzt folgenden Unterschied zu der Lösung der Normalengleichung

In [ ]:
print(np.linalg.norm(x_h - x_L))

Ähnlich können wir die QR-Zerlegung mit Givens-Rotationen angehen

In [ ]:
from scripts.Givens import qr_givens

A = np.array([[1, -1 / 4, 1 / 16],
              [1, 1 / 2,  1 / 4],
              [1, 2,      4],
              [1, 5 / 2,  25 / 4]], dtype=np.half)
b = np.array([0, 1, 0, 1], dtype=np.half)

QT = qr_givens(A)
bt = np.dot(QT, b)[:A.shape[1]]
x_g = rueckwaerts_einsetzen(A[:A.shape[1],:], bt)
print(f'x = {x_g}')

Daraus ergibt sich der Defekt

In [ ]:
print('||Ax - b|| = ', np.linalg.norm(np.inner(A2, x_g) - b))

und der Unterschied zur Lösung mithilfe der QR-Zerlegung mit Householder-Transformationen

In [ ]:
print(np.linalg.norm(x_h - x_g))